### MCP Server Usage (SSE and streamable-http)

This notebook showcases how to use the NVIDIA RAG MCP server via MCP transports instead of REST APIs. It covers:
- Launching the server (SSE and streamable-http)
- Connecting with the MCP Python client
- Listing tools
- Calling all MCP tools: `generate`, `search`, and `get_summary`

You can execute each cell in sequence to test the MCP server tools.

#### 1. Install Dependencies

Purpose:
Install the libraries needed to run the MCP server and client locally in this notebook environment.

- Ensure your environment has:
  - `mcp`, `anyio`, `httpx`, `httpx-sse`, `uvicorn`
- If using Workbench/docker, these may already be installed.

In [ ]:
# %pip install -qq -r ../nvidia_rag_mcp/requirements.txt


Note: you may need to restart the kernel to use updated packages.


INFO:     127.0.0.1:36776 - "GET /sse HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:36792 - "GET /sse HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:54070 - "GET /sse HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:54072 - "GET /sse HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:54074 - "GET /sse HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:54080 - "GET /sse HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:54082 - "GET /sse HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:54088 - "GET /sse HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:54092 - "GET /sse HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:54102 - "GET /sse HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:54112 - "GET /sse HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:54116 - "GET /sse HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:42574 - "GET /sse HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:42578 - "GET /sse HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:42580 - "GET /sse HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:42586 - "GET /sse HTTP/1.1" 404 Not Found
INFO:   

#### 2. Setup Base Configuration

Purpose:
Capture API keys and environment variables that the server and client will rely on.

Configure keys and basic variables used by the rest of this notebook.


In [ ]:
import os

NVIDIA_API_KEY = os.environ.get("NVIDIA_API_KEY", "")

#### Prerequisite: Create a collection and upload a sample document (with summary)

Purpose: Seed the knowledge base so search, generate, and get_summary produce meaningful results. We’ll upload `data/multimodal/woods_frost.pdf` with `generate_summary=true` after creating a collection.

Notes:
- Ensure the RAG server is running and reachable before using MCP tools. 
- Set `VITE_API_CHAT_URL`. If not set, using the default `http://127.0.0.1:8081`.
- Ensure the Ingestor server is running and reachable.
- Set `INGESTOR_URL`. If not set, using the default `http://127.0.0.1:8082`.
- This step creates/uses the collection `my_collection` used later in the MCP calls.

In [ ]:
# Upload a document to the Ingestor with generate_summary=True
import os
import json
import subprocess

INGESTOR_URL = os.environ.get("INGESTOR_URL", "http://127.0.0.1:8082").rstrip("/")
COLLECTION = "my_collection"
PDF_PATH = os.path.join(os.path.dirname(os.getcwd()), "data", "multimodal", "woods_frost.pdf")

# 1) Ensure the collection exists
create_cmd = [
    "curl", "-sS", "-X", "POST", f"{INGESTOR_URL}/v1/collections",
    "-H", "Content-Type: application/json",
    "-d", json.dumps([COLLECTION]),
]
print("Creating collection:", " ".join(create_cmd))
create_res = subprocess.run(create_cmd, capture_output=True, text=True)
print(create_res.stdout or create_res.stderr)

# 2) Upload document with summary generation enabled
payload = {
    "collection_name": COLLECTION,
    "blocking": True,
    "custom_metadata": [],
    "generate_summary": True,
}

cmd = [
    "curl", "-sS", "-X", "POST", f"{INGESTOR_URL}/v1/documents",
    "-F", f"documents=@{PDF_PATH}",
    "-F", f"data={json.dumps(payload)}",
]
print("Uploading document:", " ".join(cmd))
res = subprocess.run(cmd, capture_output=True, text=True)
print(res.stdout or res.stderr)

## Launch MCP Server (SSE)

Purpose:
Start the MCP server locally over SSE so that the client can connect and call tools.

This launches the MCP server on `http://127.0.0.1:8000`.

In [ ]:
# Kill any process listening on port 8000 to avoid port conflicts
import subprocess

PORT = 8000
print(f"Kill any process running on port {PORT} to start sse server in the next cell")

# Try fuser first (common on Linux)
try:
    subprocess.run(["fuser", "-k", f"{PORT}/tcp"], check=False)
except FileNotFoundError:
    print("'fuser' not found, skipping fuser-based cleanup.")
except Exception as e:
    print(f"Error while running fuser: {e}")

In [ ]:
import os
import sys
import atexit
import time

sse_proc = None
try:
    env = dict(os.environ)
    if NVIDIA_API_KEY:
        env.setdefault("NVIDIA_API_KEY", NVIDIA_API_KEY)

    repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
    server_path = os.path.join(repo_root, "nvidia_rag_mcp", "mcp_server.py")
    cmd = [
        sys.executable,
        server_path,
        "--transport",
        "sse",
        "--host",
        "127.0.0.1",
        "--port",
        "8000",
    ]

    print("Launching:", " ".join(cmd))
    sse_proc = subprocess.Popen(cmd, env=env)
    atexit.register(lambda: sse_proc and sse_proc.poll() is None and sse_proc.terminate())
    time.sleep(2.0)
    print("SSE server PID:", sse_proc.pid)
except Exception as e:
    print("Failed to start SSE server:", e)

## Connect with MCP Client (SSE), List Tools, and Call MCP Tools

Purpose:
Verify connectivity by listing available tools and invoking the MCP tools (`generate`, `search`, `get_summary`) using the SSE transport.

This uses the `mcp_client.py` CLI to connect over SSE, list tools, and invoke the RAG tools.

**Note:** Ensure the SSE server is running from Cell 6 before executing this cell.


In [ ]:
import os
import sys
import json
import subprocess

SSE_URL = "http://127.0.0.1:8000/sse"
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
client_path = os.path.join(repo_root, "nvidia_rag_mcp", "mcp_client.py")

print("="*80)
print("Listing available tools...")
print("="*80)
subprocess.run([
    sys.executable,
    client_path,
    "list",
    "--transport=sse",
    f"--url={SSE_URL}",
])

print("\n" + "="*80)
print("Calling 'generate' tool...")
print("="*80)
generate_args = json.dumps({
    "messages": [{"role": "user", "content": "Hello from SSE demo"}],
    "collection_name": "my_collection",
})
subprocess.run([
    sys.executable,
    client_path,
    "call",
    "--transport=sse",
    f"--url={SSE_URL}",
    "--tool=generate",
    f"--json-args={generate_args}",
])

print("\n" + "="*80)
print("Calling 'search' tool...")
print("="*80)
search_args = json.dumps({
    "query": "Tell me about Robert Frost's poems",
    "collection_name": "my_collection",
    "reranker_top_k": 2,
    "vdb_top_k": 5,
    "enable_query_rewriting": False,
    "enable_reranker": True,
})
subprocess.run([
    sys.executable,
    client_path,
    "call",
    "--transport=sse",
    f"--url={SSE_URL}",
    "--tool=search",
    f"--json-args={search_args}",
])

print("\n" + "="*80)
print("Calling 'get_summary' tool...")
print("="*80)
summary_args = json.dumps({
    "collection_name": "my_collection",
    "file_name": "woods_frost.pdf",
    "blocking": False,
    "timeout": 60,
})
subprocess.run([
    sys.executable,
    client_path,
    "call",
    "--transport=sse",
    f"--url={SSE_URL}",
    "--tool=get_summary",
    f"--json-args={summary_args}",
])


## Launch MCP Server (streamable_http)

Purpose:
Start the MCP server locally using the **streamable_http** transport so that the client can connect and call tools.

This launches the MCP server with FastMCP's streamable-http support.

In [ ]:
PORT = 8000
print(f"Kill any process running on port {PORT} to start streamable_http server in the next cell")

try:
    subprocess.run(["fuser", "-k", f"{PORT}/tcp"], check=False)
except FileNotFoundError:
    print("'fuser' not found, skipping fuser-based cleanup.")
except Exception as e:
    print(f"Error while running fuser: {e}")

In [ ]:
import os
import sys
import atexit
import time

stream_proc = None
try:
    env = dict(os.environ)
    if NVIDIA_API_KEY:
        env.setdefault("NVIDIA_API_KEY", NVIDIA_API_KEY)

    repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
    server_path = os.path.join(repo_root, "nvidia_rag_mcp", "mcp_server.py")
    cmd = [
        sys.executable,
        server_path,
        "--transport",
        "streamable_http",
    ]

    print("Launching streamable_http server:", " ".join(cmd))
    stream_proc = subprocess.Popen(cmd, env=env)
    atexit.register(lambda: stream_proc and stream_proc.poll() is None and stream_proc.terminate())
    time.sleep(2.0)
    print("streamable_http server PID:", stream_proc.pid)
except Exception as e:
    print("Failed to start streamable_http server:", e)

## Connect with MCP Client (streamable_http), List Tools, and Call MCP Tools

Purpose:
Verify connectivity by listing available tools and invoking the MCP tools (`generate`, `search`, `get_summary`) using the streamable_http transport.

This uses the `mcp_client.py` CLI to connect over streamable_http, list tools, and invoke the RAG tools.

**Note:** Ensure the streamable_http server is running from the cell above before executing this cell.



In [29]:
import os
import sys
import json
import subprocess

SSE_URL = "http://127.0.0.1:8000/mcp"
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
client_path = os.path.join(repo_root, "nvidia_rag_mcp", "mcp_client.py")

print("="*80)
print("Listing available tools...")
print("="*80)
subprocess.run([
    sys.executable,
    client_path,
    "list",
    "--transport=streamable_http",
    f"--url={SSE_URL}",
])

print("\n" + "="*80)
print("Calling 'generate' tool...")
print("="*80)
generate_args = json.dumps({
    "messages": [{"role": "user", "content": "Hello from SSE demo"}],
    "collection_name": "my_collection",
})
subprocess.run([
    sys.executable,
    client_path,
    "call",
    "--transport=streamable_http",
    f"--url={SSE_URL}",
    "--tool=generate",
    f"--json-args={generate_args}",
])

print("\n" + "="*80)
print("Calling 'search' tool...")
print("="*80)
search_args = json.dumps({
    "query": "Tell me about Robert Frost's poems",
    "collection_name": "my_collection",
    "reranker_top_k": 2,
    "vdb_top_k": 5,
    "enable_query_rewriting": False,
    "enable_reranker": True,
})
subprocess.run([
    sys.executable,
    client_path,
    "call",
    "--transport=streamable_http",
    f"--url={SSE_URL}",
    "--tool=search",
    f"--json-args={search_args}",
])

print("\n" + "="*80)
print("Calling 'get_summary' tool...")
print("="*80)
summary_args = json.dumps({
    "collection_name": "my_collection",
    "file_name": "woods_frost.pdf",
    "blocking": False,
    "timeout": 60,
})
subprocess.run([
    sys.executable,
    client_path,
    "call",
    "--transport=streamable_http",
    f"--url={SSE_URL}",
    "--tool=get_summary",
    f"--json-args={summary_args}",
])


Listing available tools...


[12/04/25 13:23:59] INFO     Created new          ]8;id=330024;file:///home/niyati/anaconda3/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=447520;file:///home/niyati/anaconda3/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             ac1e4821d04f45238b05                               
                             1088df19b561                                       
                    INFO     Processing request of type            ]8;id=187297;file:///home/niyati/anaconda3/lib/python3.13/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=839178;file:///home/niyati/anaconda3/lib/python3.13/site-packages/mcp/server/lowlevel/server.py#709\709]8;;\
                             ListToolsRequest 

INFO:     127.0.0.1:50702 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50704 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:50710 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50714 - "POST /mcp HTTP/1.1" 200 OK
generate: Generate an answer using NVIDIA RAG (optionally with knowledge base). Provide chat messages and optional generation parameters.
search: Search the vector database and return citations for a given query.
get_summary: Retrieve the pre-generated summary for a document from a collection. Set blocking=true to wait up to timeout seconds for summary generation.
INFO:     127.0.0.1:50726 - "DELETE /mcp HTTP/1.1" 200 OK

Calling 'generate' tool...


                    INFO     Created new          ]8;id=542499;file:///home/niyati/anaconda3/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=277821;file:///home/niyati/anaconda3/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             1f40ea48c84546228fa4                               
                             e414a531657d                                       
                    INFO     Processing request of type            ]8;id=870457;file:///home/niyati/anaconda3/lib/python3.13/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=66015;file:///home/niyati/anaconda3/lib/python3.13/site-packages/mcp/server/lowlevel/server.py#709\709]8;;\
                             CallToolRequest   

INFO:     127.0.0.1:50732 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50736 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:50750 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50760 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50772 - "POST /mcp HTTP/1.1" 200 OK
{
  "meta": null,
  "content": [
    {
      "type": "text",
      "text": "Hello!",
      "annotations": null,
      "meta": null
    }
  ],
  "structuredContent": {
    "result": "Hello!"
  },
  "isError": false
}
INFO:     127.0.0.1:50774 - "DELETE /mcp HTTP/1.1" 200 OK

Calling 'search' tool...


...

INFO:     127.0.0.1:50834 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50850 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:50864 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50878 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:50894 - "POST /mcp HTTP/1.1" 200 OK
{
  "meta": null,
  "content": [
    {
      "type": "text",
      "text": "{\n  \"message\": \"Summary retrieved successfully.\",\n  \"summary\": \"Here is the concise summary:\\n\\n**Stopping by Woods on a Snowy Evening, Poem by Robert Frost**\\nThe poem describes a serene winter scene where the narrator stops to observe woods filling with snow, prompting reflection on duties and journeys ahead. Key themes include the allure of nature, responsibility, and the passage of time. Notable lines highlight the contrast between the woods' beauty and the narrator's obligations: \\\"The woods are lovely, dark and deep, But I have promises to keep, And miles to go before I sleep.\\\" The poem was published as part 

                    INFO     Created new          ]8;id=732849;file:///home/niyati/anaconda3/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=787229;file:///home/niyati/anaconda3/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             b70a604d68b74825a1a5                               
                             bd46e1e6d7ce                                       
                    INFO     Processing request of type            ]8;id=632272;file:///home/niyati/anaconda3/lib/python3.13/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=589165;file:///home/niyati/anaconda3/lib/python3.13/site-packages/mcp/server/lowlevel/server.py#709\709]8;;\
                             CallToolRequest  

CompletedProcess(args=['/home/niyati/anaconda3/bin/python', '/home/niyati/rag/nvidia_rag_mcp/mcp_client.py', 'call', '--transport=streamable_http', '--url=http://127.0.0.1:8000/mcp', '--tool=get_summary', '--json-args={"collection_name": "my_collection", "file_name": "woods_frost.pdf", "blocking": false, "timeout": 60}'], returncode=0)

[12/04/25 13:26:51] INFO     Created new          ]8;id=298045;file:///home/niyati/anaconda3/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=313609;file:///home/niyati/anaconda3/lib/python3.13/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             dc2c33348f6b49dd92db                               
                             cd336dcb47b2                                       
                    INFO     Processing request of type            ]8;id=794346;file:///home/niyati/anaconda3/lib/python3.13/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=845500;file:///home/niyati/anaconda3/lib/python3.13/site-packages/mcp/server/lowlevel/server.py#709\709]8;;\
                             ListToolsRequest 

INFO:     127.0.0.1:53774 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:53786 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:53798 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:53812 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:53824 - "DELETE /mcp HTTP/1.1" 200 OK


## Cleanup & Troubleshooting

Purpose:
Wrap up the session, stop background processes, and provide guidance for common errors (401/404) and environment/version mismatches.

- To stop the SSE or streamable-http server started above, restart the kernel or run the cell that terminates the `sse_proc`.
- If SSE returns 404, ensure you're connecting to the base URL (the client probes standard SSE endpoints).
- Ensure versions of `mcp`, `anyio`, and `uvicorn` match your environment constraints.
